<a href="https://colab.research.google.com/github/Zsupi/Counter/blob/main/HF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning a Vizuális Informatikában
##2. Házi Feladat

###1. Rész

Valósíts meg egy paraméterezhető konvolúciós neurális hálózatot, amely osztályozásra képes. A hálózat paraméterei a következők:


*   nC: Az osztályok száma
*   nFeat: Az első réteg kimeneti csatornaszáma. Az ezt követő rétegek be- és kimeneti csatornaszáma egyezzen ezzel meg, majd minden leskálázó (strided konvolúciós) réteg duplázza ezt meg.
*   nLevels: A háló szintjeinek száma. Egy szintnek az azonos térbeli kiterjedésű tenzorokon operáló rétegeket nevezzük (2 leskálázás közt). (Tipp: használj adaptív poolingot az osztályozó réteg előtt, hogy a változó szint szám ne okozzon problémát.)
*   layersPerLevel: Az egy szinten található konvolúciós rétegek száma.
*   kernelSize: A konvolúciós rétegek mérete.
*   nLinType: Kategorikus változó, amellyel a nemlinearitás típusát állíthatja (hogy hány és milyen függvények közül lehet választani önre van bízva)
*   bNorm: Bináris változó, amellyel állítható, hogy a konvolúciós rétegekbe teszünk-e BatchNormot.
*   residual: Bináris változó, True érték esetén minden szinten valósíts meg egy reziduális kapcsolatot a szint bemenete és a szint végén megjelenő leskálázó réteg bemenete közt.

Tipp: Érdemes ehhez írni először két külön modult, ami egy réteget (batchnormmal, dropouttal és nemlinearitással) valósít meg és egyet, ami meg ezekből egy szintet legózik össze. Ezekből aránylag könnyen összelegózható a háló.

Tipp 2: A PyTorchnak van nn.Sequential osztálya. Ez egy lista, amiben rétegek vannak, de ha sima lista változóba tesztek egyszerre több nn.Module-t, annak az optimizer nem fogja megkapni. Ez azért van, mert amikor egy nn.Module-tól leszármazó osztálytól elkéritek a .parameters()-t, akkor az végignézi az objektum összes tagváltozót, hogy van-e neki .parameters() függvénye (és ezt szépen rekurzívan végigcsinálja). A Listának pedig nincs, hiába vannak benne olyan elemek, amiknek van. Ezen felül az nn.Sequential forward függvénye is felül van csapva, és szépen sorban meghívja a belül lévő rétegeket.



In [75]:
import torch
import torch.nn as nn

class Layer(nn.Module):
  def __init__(self, ich, och, ks, stride=1, padding=1, bNorm=True, nLinType='relu'):
    super(Layer, self).__init__()
    opt = []

    c = nn.Conv2d(ich, och, ks, stride=stride, padding=ks//2)
    opt.append(c)
    if bNorm:
      opt.append(nn.BatchNorm2d(och))
    if nLinType == 'relu':
      opt.append(nn.ReLU())
    if nLinType == 'leakyrelu':
      opt.append(nn.LeakyReLU(0.2))

    self.opt = nn.Sequential(*opt)

  def forward(self, x):
    return self.opt(x)


class Level(nn.Module):
  def __init__(self, ch, nLevels, layersPerLevel, ks, bNorm=True, nLinType='relu', residual=False):
    super(Level, self).__init__()
    layers = []
    for i in range(layersPerLevel):
      layers.append(Layer(ch, ch, ks, bNorm=bNorm, nLinType=nLinType))

    self.layers = nn.Sequential(*layers)

    self.pooling = Layer(ch, ch*2, ks, bNorm=bNorm, nLinType=nLinType)
    self.residual = residual

  def forward(self, x):
    result = self.layers(x)
    return self.pooling(result)

class Net(nn.Module):
  def __init__(self, nC, nFeat, nLevels, layersPerLevel, ks, nLinType='relu', bNorm=True, ):
    super(Net, self).__init__()
    levels = []
    self.zero = nn.Conv2d(3, nFeat, ks, stride=1, padding=ks//2)
    for i in range(nLevels):
      levels.append(Level(nFeat, 1, layersPerLevel, ks, bNorm, nLinType))
      nFeat *= 2
    self.levels = nn.Sequential(*levels)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.classifier = nn.Conv2d(nFeat, nC, 1)

  def forward(self, x):
    x = self.zero(x)
    x = self.levels(x)
    x = self.avgpool(x)
    return torch.squeeze(self.classifier(x))


## Generate Net

In [76]:
def net():
  nC = 10
  nFeat = 6
  nLevels = 3
  layersPerLevel = 2
  kernelSize = 3
  nLinType = 'relu'
  bNorm = True

  bSize = 32
  lr = 0.001
  lr_ratio = 0.1
  numEpoch = 10
  decay = 0.0001
  return Net(nC, nFeat, nLevels, layersPerLevel, kernelSize, nLinType, bNorm)

###2. Rész

Valósíts meg egy paraméterezhető neurális háló tanító függvényt.

A függvény bemenetként megkapja a fenti neurális háló megkostruálásához szükséges paramétereket, az összes random seedet 42 értékre állítja, majd végrehajtja a neurális háló tanítását a kiadott adatbázison.

Használj Adam optimizert és Cosine Annealing tanulási ráta ütemezőt.

A tanítás során minden epoch után validálj, és jegyezd fel a legjobb validációs pontosságot, és a tanítás végén ezt add vissza.

A függvénynek további bemeneti paraméterei:

*   bSize: A bacth méret
*   lr: a tanulási ráta
*   lr_ratio: a tanulási ráta ütemező eta_min paramétere és a kezdeti tanulási ráta hányadosa
*   numEpoch: az epochok száma
*   decay: a weight_decay paraméter értéke



In [77]:
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from PIL import Image
import torch.optim as optim

transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding = 4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.25,0.25,0.2,0.1),
        transforms.ToTensor(), #/255 toFloat()
        transforms.transforms.Normalize(
            [0.5,0.5,0.5],
            [0.25,0.25,0.25]
        )
    ]
)


transform_val = transforms.Compose(
    [
        transforms.ToTensor(), #/255 toFloat()
        transforms.transforms.Normalize(
            [0.5,0.5,0.5],
            [0.25,0.25,0.25]
        )
    ]
)

dataset = dset.ImageFolder(root="Small/Classification/train/", transform=transform)
dataset_val = dset.ImageFolder(root="Small/Classification/train/", transform=transform_val)

trainLoader = DataLoader(dataset, batch_size =128, shuffle = True, num_workers = 2)
trainLoader_val = DataLoader(dataset_val, batch_size =128, shuffle = True, num_workers = 2)

## Train

In [82]:
def train(epoch, optimizer, net):

  running_loss = 0.0
  correct = 0.0
  total = 0

  # set the network to train (for batchnorm and dropout)
  net.train()

  # Create progress bar
  bar = display(progress(0, len(dataset)), display_id=True)
  criterion = nn.CrossEntropyLoss()

  # Epoch loop
  for i, data in enumerate(trainLoader, 0):
    # get the inputs
    inputs, labels = data

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # compute statistics
    running_loss += loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    # Update progress bar
    bar.update(progress(i+1, len(trainLoader)))

  # print and plot statistics
  tr_loss = running_loss / len(trainLoader)
  tr_corr = correct / total
  print("Epoch [%d] Training Loss: %.4f Training Pixel Acc: %.2f" % (epoch+1, tr_loss, tr_corr))

  return tr_loss,tr_corr

## Validate

In [83]:
def val(epoch, numClass, optimizer, net):

  # variables for loss
  running_loss = 0.0
  correct = 0.0
  total = 0

  # IoU computation
  conf = torch.zeros(numClass,numClass)
  IoU = torch.zeros(numClass)
  labCnts = torch.zeros(numClass)
  criterion = nn.CrossEntropyLoss()

  # set the network to eval (for batchnorm and dropout)
  net.eval()

  # Create progress bar
  bar = display(progress(0, len(trainLoader_val)), display_id=True)

  # Epoch loop
  for i, data in enumerate(trainLoader_val, 0):
    # get the inputs
    inputs, labels = data

    # forward
    outputs = net(inputs)
    loss = criterion(outputs, labels)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    running_loss += loss.cpu().item()


    tr_loss = running_loss / len(trainLoader_val)
    tr_acc = correct/total*100

    return tr_loss, tr_acc

## Epoch

In [86]:

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def train_neural_network(nC, nFeat, nLevels, layersPerLevel, kernelSize, nLinType, bNorm, bSize, lr, lr_ratio, numEpoch,  random_seed=42):

  net = Net(nC, nFeat, nLevels, layersPerLevel, kernelSize, nLinType, bNorm)
  optimizer = optim.Adam(net.parameters(), lr=lr)

  eta_min = lr * lr_ratio
  scheduler = CosineAnnealingLR(optimizer, T_max=numEpoch, eta_min=eta_min)

  best_val_corr = 0

  # Makes multiple runs comparable
  torch.manual_seed(random_seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  trLosses = []
  trAccs = []
  valLosses = []
  valAccs = []

  for epoch in range(numEpoch):

    # Call train and val
    tr_loss,tr_corr = train(epoch, optimizer, net)
    val_loss,val_corr = val(epoch, 10, optimizer, net)

    trLosses.append(tr_loss)
    trAccs.append(tr_corr)
    valLosses.append(val_loss)
    valAccs.append(val_corr)

    # Step with the scheduler
    scheduler.step()

    if val_corr > best_val_corr:
      best_val_corr = val_corr
      torch.save(net, 'model.pt')
      print("Best model")

  return best_val_corr

### Adatbázis letöltése

In [69]:
!wget http://deeplearning.iit.bme.hu/Public/Small.zip --no-check-certificate
!unzip -qq Small.zip
!rm Small.zip

--2024-05-04 14:04:31--  http://deeplearning.iit.bme.hu/Public/Small.zip
Resolving deeplearning.iit.bme.hu (deeplearning.iit.bme.hu)... 152.66.243.112
Connecting to deeplearning.iit.bme.hu (deeplearning.iit.bme.hu)|152.66.243.112|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://deeplearning.iit.bme.hu/Public/Small.zip [following]
--2024-05-04 14:04:33--  https://deeplearning.iit.bme.hu/Public/Small.zip
Connecting to deeplearning.iit.bme.hu (deeplearning.iit.bme.hu)|152.66.243.112|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 7048374 (6.7M) [application/zip]
Saving to: ‘Small.zip’

Small.zip           100%[===================>]   6.72M  5.20MB/s    in 1.3s    

2024-05-04 14:04:34 (5.20 MB/s) - ‘Small.zip’ saved [7048374/7048374]

replace Small/Classification/train/0/img1016.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Progress Bar

In [ ]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

###3. Rész

Valósíts meg hiperparaméter optimalizálást a Bayesian Optimization python könyvtár felhasználásával. A könyvtár itt érhető el: https://github.com/fmfn/BayesianOptimization

A megoldás során a következőkre ügyelj:


1.   A kezdeti random lépések száma legyen kb egyenlő a szabad paraméterek számával (8). A bináris és kategorikus változókat nem érdemes optimalizálni külön, és az epochszámot is érdemes fixen megválasztani.
2.   Az teljes lépésszám legyen ennek 4-szerese (32)
3.   Mivel a Bayesian Optimization függvény a bináris/diszkrét/integer paramétereket nem támogatja, ezért a folytonos értékek megfelelő konverziója az előző feladatrészben megvalósított függvény feladata. Itt külön figyeljetek arra, hogy vannak olyan változók (pl a szűrők alap száma, vagy a szűrő mérete), amiknek csak bizonyos értékek értelmesek.
4.   Az epochok közben menő progress barokat meg kiíratásokat érdemes eltüntetni, a Bayesian opt majd fog írogatni
5.   Referenciaként, az adatbázison olyan 94-5% pontosság az elfogadható és 96%+ számít jónak.
6.   Ha megvan a végső legjobb paraméter, akkor érdemes azzal egy kicsit hosszabb tanítást lefuttatni.

Hogy a paramétereket milyen tartományban akarjátok optimalizálni, azt nektek kellene kitalálni. De a szintek számát és a szűrőméretet érelmetlen 5 fölé, a rétegek számát meg 3 fölé vinni. Az epochszámmal se menjetek 20 fölé, mert így is kb 1 óráig tart egy optimalizálás.



In [73]:
!pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization

# Define the objective function for Bayesian Optimization
def bayesian_optimization(nFeat: int, nLevels: int, layersPerLevel: int, kernelSize: int, bSize, lr: float, decay):
    # Fixed hyperparameters
    nC = 10  # Number of classes
    nLinType = 'relu'
    numEpoch = 10  # Number of epochs
    kernelSize = int(kernelSize)
    nFeat = int(nFeat)
    layersPerLevel = int(layersPerLevel)
    nLevels = int(nLevels)
    bNorm = True

    return train_neural_network(nC, nFeat, nLevels, layersPerLevel, kernelSize, nLinType, bNorm, bSize, lr, 0.1, numEpoch, decay)

# Set the optimization bounds
pbounds = {
    'nFeat': (6, 12),
    'nLevels': (3, 5),
    'layersPerLevel': (1, 3),
    'kernelSize': (3, 5),
    'bSize': (32, 64),
    'lr': (1e-4, 1e-2),
    'decay':  (0.0001, 1e-3),
}

# Initialize Bayesian Optimization
opt = BayesianOptimization(f=bayesian_optimization, pbounds=pbounds, random_state=42)

# Perform Bayesian Optimization
opt.maximize(init_points=8, n_iter=24)

# Print the best parameters and corresponding accuracy
print("Best Parameters:", opt.max['params'])
print("Best Accuracy:", opt.max['target'])

|   iter    |  target   |   bSize   |   decay   | kernel... | layers... |    lr     |   nFeat   |  nLevels  |
-------------------------------------------------------------------------------------------------------------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch [1] Training Loss: 1.5974 Training Pixel Acc: 0.59


Best model


Epoch [2] Training Loss: 0.9024 Training Pixel Acc: 0.80


...

Epoch [3] Training Loss: 0.5319 Training Pixel Acc: 0.79


Epoch [4] Training Loss: 0.4794 Training Pixel Acc: 0.82


Best model


Epoch [5] Training Loss: 0.3888 Training Pixel Acc: 0.86


Epoch [6] Training Loss: 0.3468 Training Pixel Acc: 0.88
